In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
#Load SFEW data:
!rm -rf SFEW_dataset

!git clone https://github.com/Caterina1996/SFEW_dataset


Cloning into 'SFEW_dataset'...
remote: Enumerating objects: 1345, done.
remote: Counting objects: 100% (1345/1345), done.
remote: Compressing objects: 100% (1332/1332), done.
remote: Total 1345 (delta 18), reused 1328 (delta 13), pack-reused 0
Receiving objects: 100% (1345/1345), 41.10 MiB | 29.27 MiB/s, done.
Resolving deltas: 100% (18/18), done.


In [ ]:
%%capture
! unzip /content/gdrive/MyDrive/EmotiW_2018.zip

In [ ]:
#Unzip faces frames and change folder names
%%capture 
! unzip /content/EmotiW_2018/Val_AFEW/AlignedFaces_LBPTOP_Points_Val.zip
!mv /content/AlignedFaces_LBPTOP_Points_Val /content/Val_faces_videos


!unzip /content/EmotiW_2018/Train_AFEW/AlignedFaces_LBPTOP_Points.zip
!mv AlignedFaces_LBPTOP_Points Train_faces_videos

In [ ]:
import operator
import keras
import numpy as np
from numpy.random import seed
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Conv2D, BatchNormalization, MaxPooling2D
from keras import applications

from keras.models import load_model
from keras.callbacks import ModelCheckpoint,CSVLogger

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import *
import sklearn.metrics as sk_metrics
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

import glob
import ntpath as nt
import os
from PIL import Image
import operator
from keras import optimizers
from shutil import copyfile
from shutil import copyfile
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img

from PIL import Image
from sklearn.metrics import f1_score
import tensorflow as tf
from keras import applications
import copy
import collections
import pickle
# https://riptutorial.com/keras/example/32608/transfer-learning-using-keras-and-vgg

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
from keras.models import Model

#fix weird vgg problem:
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
#Set seed
seed(42)
tf.random.set_seed(42)

In [ ]:
#Parameters initializations:

img_width, img_height =128, 128 #revisar Las originales son 143 x 181
batch_size=2
learning_rate = 0.00001
epochs = 100
batch_size = 2
architecture='VGG16'
img_width, img_height=img_width, img_height
num_classes=7
dropout = 0.25
dataugmentation=True

train_dir='SFEW_dataset/Train'
test_dir='SFEW_dataset/Val'

emotions_list=os.listdir("/content/SFEW_dataset/Train")
inidices=list(range(0,7))

emotions_dict=dict(zip(inidices,emotions_list))
emotions_reversed_dict = {v: k for k, v in emotions_dict.items() }

In [ ]:
emotions_dict

{0: 'Fear',
 1: 'Sad',
 2: 'Surprise',
 3: 'Happy',
 4: 'Neutral',
 5: 'Disgust',
 6: 'Angry'}

In [ ]:
def create_labels_set_dir(set_path):
  video_labels={}
  for emotion in emotions_list:
    for video_id in os.listdir(set_path+emotion):
      video_labels[video_id.split(".")[0]]=emotion
  return video_labels

In [ ]:
def get_emo(prediction):
  emo=list(prediction).index(max(prediction))
  return emotions_dict[emo]

def get_emo(prediction):
  emo=list(prediction).index(1)
  return emotions_dict[emo]

def get_emo_value(prediction):
  return list(prediction).index(max(prediction))

emotions_dict


{0: 'Fear',
 1: 'Sad',
 2: 'Surprise',
 3: 'Happy',
 4: 'Neutral',
 5: 'Disgust',
 6: 'Angry'}

In [ ]:
val_dir="/content/EmotiW_2018/Val_AFEW/"
val_video_labels=create_labels_set_dir(val_dir)

train_dir="/content/EmotiW_2018/Train_AFEW/"
train_video_labels=create_labels_set_dir(train_dir)


In [ ]:
print(len(val_video_labels))
print(len(train_video_labels))

381
773


In [ ]:
#save afew labels
a_file = open("/content/gdrive/MyDrive/TFM_MUSI/combined_model/train_video_labels.pkl", "wb")

pickle.dump(train_video_labels, a_file)

a_file.close()

a_file = open("/content/gdrive/MyDrive/TFM_MUSI/combined_model/val_video_labels.pkl", "wb")

pickle.dump(val_video_labels, a_file)

a_file.close()

In [ ]:
for emotion in emotions_list:
  if not os.path.exists("SFEW_images_videos/Train/"+emotion):
    os.makedirs("SFEW_images_videos/Train/"+emotion)

  if not os.path.exists("EMOTIW_test_Faces/"+emotion):
    os.makedirs("SFEW_images_videos/Val/"+emotion)



In [ ]:
#group sfew images acording to their video label
def tidy_SFEW(origin_dir,destination_dir,emo_video_labels):

  for emo in emotions_list:
    if not os.path.exists(destination_dir+emo):
      print("HI")
      os.makedirs(destination_dir+emo)

  # obtener la emocion del video a la que pertenecen las imagenes del sfew
  val_images=glob.glob(origin_dir+"*/*/*")
  val_images
  i=0
  print("dir0",val_images[0])
  for image_path in val_images:
    print("image path:",image_path)

    image_video_id=image_path.split("/")[-1].split("_")[-2]
    image_id=image_path.split("/")[-1].split("_")[-1]
    sfew_emo=image_path.split("/")[-2]

    print("video_id:",image_video_id)
    print("image_id:",image_id)
    print("sfew_emo:",sfew_emo)

    try:
      afew_emo=emo_video_labels[image_video_id]
      destination=destination_dir+afew_emo+"/"+image_video_id
      print("destination_dir: ",destination)

      if not os.path.exists(destination):
        os.makedirs(destination)
      copyfile(image_path,destination+"/"+image_id)
      if sfew_emo != afew_emo:
        print("SFEW emo and afew emo dont match")
        print("SFEW emo was :", sfew_emo)
        print("AFEW emo is :", afew_emo)
        # display 
        im = Image.open(image_path)
        plt.imshow(im)
        plt.show()
        i+=1
        print("number of discrepances is", i)

    except Exception as e: 
      print(e)
      print("id not found: ",image_video_id)

print("TRAIN---------------------------------------------------------------------------")
origin_dir="/content/SFEW_dataset/Train"
destination_dir="/content/SFEW_images_videos/Train/"
tidy_SFEW(origin_dir,destination_dir,train_video_labels)

print("TEST---------------------------------------------------------------------------")
origin_dir="/content/SFEW_dataset/Val"
destination_dir="/content/SFEW_images_videos/Val/"
tidy_SFEW(origin_dir,destination_dir,val_video_labels)



In [ ]:
#Load static model Vgg16 with last block retrained with SFEW
#We will use this model to predict frame emotions of the video
model=keras.models.load_model("/content/gdrive/MyDrive/TFM_MUSI/VGGmodel/SFEW_corrected/SFEW_TF_corrected0.h5")

In [ ]:
#Analize model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [ ]:
#Create sets


train_data_dir='SFEW_dataset/Train'
test_data_dir='SFEW_dataset/Val'

# train_set,test_set=create_sets(dataugmentation,train_dir,test_dir,batch_size)

train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   brightness_range=[0.5,1.0],
                                   zoom_range=0.1,
                                   shear_range=0.2,
                                  #  width_shift_range=[-40,40]
                                  #  height_shift_range=0.1,
                                   validation_split=0.2,
                                   horizontal_flip=True)
                                  #  height_shift_range=0.1

test_datagen= ImageDataGenerator(rescale=1. / 255)

train_set = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    # color_mode='grayscale',
    subset='training',
    shuffle=True)

# val_set = train_datagen.flow_from_directory(
#     train_data_dir, # same directory as training data
#     target_size=(img_height, img_width),
#     batch_size=batch_size,
#     class_mode='categorical',
#     subset='validation') 
  
test_set = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    # color_mode='grayscale',
    shuffle=True)

nb_train_samples = len(train_set.filenames)
nb_test_samples = len(test_set.filenames)
# nb_val_samples = len(val_set.filenames)

train_set.class_indices
emotions_dict = {v: k for k, v in train_set.class_indices.items()}
print(emotions_dict)

In [ ]:
#For each video compute predictions of video frames using the tuned vgg16 model
#already trained on static imgs and average the video frames predictions to obtain
#a video predictions vector

#Saves video predictions in a dictionary where keys are video labels and values are video predictions


from sklearn.metrics import confusion_matrix, accuracy_score
def evaluate_videos_score(videos_paths,model):
  # videos_paths=glob.glob("/content/SFEW_images_videos/*/*")
  y_real=[]
  y_pred=[]
  predictions=[]
  set_preds_dict={}
  j=0

  for video_path in videos_paths:
    print(video_path)
    prediction=np.zeros(7)
    video_images=glob.glob(video_path+"/*")
    video_id=video_path.split("/")[-1]
    # print(video_images)
    n=len(video_images)
    print("n is: ",n)
    i=1

    for image_path in video_images:
      img=tf.keras.preprocessing.image.load_img(image_path, color_mode="rgb",target_size=(128,128))
      x = img_to_array(img)
      x=x/255.
      # new_x=tf.expand_dims(x, axis=0)
      input = np.array([x]) 
      y=model.predict(input)
      # print(y)
      
      prediction=prediction+y
      # print("i is: ",i)
      # print("n is: ",n)
      try:
        if i==n:
          print("video num :",j)
          j+=1
          prediction=prediction/n  
          predictions.append(prediction)
          set_preds_dict[video_id]=prediction

          print("img_path is:",image_path)
          # real_label=image_path.split("/")[4]
          real_label=image_path.split("/")[3]
          print("real label is: ",real_label)
          pred_label=get_emo_value(prediction[0])
          y_real.append(train_set.class_indices[real_label])
          y_pred.append(pred_label)

          if real_label!=get_emo(prediction[0]):
            print("labels don't match!!!!!!----------------------------------------------------")
            print("Real label:",real_label)
            print("Predicted emo:",get_emo(prediction[0]))
            print(emotions_dict)
            print("prediction from video:",prediction[0])

            # im = Image.open(image_path)
            # plt.imshow(im)
            # plt.show()
      except Exception as e:
        print(e)
      i+=1

  print(confusion_matrix(y_real,y_pred))
  print(accuracy_score(y_real, y_pred))
  return y_real,y_pred,predictions,set_preds_dict


# videos_paths=glob.glob("/content/SFEW_images_videos/Train/*/*")
# y_real_sfew_tr,y_pred_sfew_tr=evaluate_videos_score(videos_paths)

In [ ]:
#compute predictions
videos_paths=glob.glob("/content/SFEW_images_videos/Train/*/*")
y_real_sfew_tr,y_pred_sfew_tr,predictions_tr,sfew_predictions_tr=evaluate_videos_score(videos_paths,model)

videos_paths=glob.glob("/content/SFEW_images_videos/Val/*/*")
y_real_sfew_te,y_pred_sfew_te,predictions_te,sfew_predictions_te=evaluate_videos_score(videos_paths,model)

In [ ]:
#Save predictions
import pickle

a_file = open("/content/gdrive/MyDrive/TFM_MUSI/combined_model/sfew_predictions_tr.pkl", "wb")

pickle.dump(sfew_predictions_tr, a_file)

a_file.close()

a_file = open("/content/gdrive/MyDrive/TFM_MUSI/combined_model/sfew_predictions_te.pkl", "wb")

pickle.dump(sfew_predictions_tr, a_file)

In [ ]:
#Check that it is saved correctly
a_file = open("/content/gdrive/MyDrive/TFM_MUSI/combined_model/sfew_predictions_tr.pkl", "rb")

output = pickle.load(a_file)

print(output)

In [ ]:
#Test con imgs frames del afew (Discarded)

In [ ]:
#Create new sorted by label folders where we store video frames faces according to their video emotion label:
!rm -rf EMOTIW_val_Faces
!rm -rf EMOTIW_train_Faces
!rm -rf EMOTIW_test_Faces

!mkdir EMOTIW_train_Faces
!mkdir EMOTIW_val_Faces
!mkdir EMOTIW_test_Faces

In [ ]:
for emotion in emotions_list:
  if not os.path.exists("EMOTIW_train_Faces/"+emotion):
    os.makedirs("EMOTIW_train_Faces/"+emotion)

  if not os.path.exists("EMOTIW_val_Faces/"+emotion):
    os.makedirs("EMOTIW_val_Faces/"+emotion)

  if not os.path.exists("EMOTIW_test_Faces/"+emotion):
    os.makedirs("EMOTIW_test_Faces/"+emotion)

In [ ]:
train_data_dir='SFEW_dataset/Train'
test_data_dir='SFEW_dataset/Val'

# train_set,test_set=create_sets(dataugmentation,train_dir,test_dir,batch_size)

train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   brightness_range=[0.5,1.0],
                                   zoom_range=0.1,
                                   shear_range=0.2,
                                  #  width_shift_range=[-40,40]
                                  #  height_shift_range=0.1,
                                   validation_split=0.2,
                                   horizontal_flip=True)
                                  #  height_shift_range=0.1

test_datagen= ImageDataGenerator(rescale=1. / 255)

train_set = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    # color_mode='grayscale',
    subset='training',
    shuffle=True)

val_set = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') 
  
test_set = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    # color_mode='grayscale',
    shuffle=True)

nb_train_samples = len(train_set.filenames)
nb_test_samples = len(test_set.filenames)
nb_val_samples = len(val_set.filenames)

train_set.class_indices
emotions_dict = {v: k for k, v in train_set.class_indices.items()}
print(emotions_dict)

predictions=evaluate_model(model)

In [ ]:
#tidy aligned faces from afew in train and test sets
def tidy_set(origin_faces_paths,dest_dir,dict_set,version1=False,videos_val_dir="/content/EMOTIW_val_Faces/",test=False):
  i=0
  j=0
  j_fotos,k_fotos=0,0
  k=0
  destination_dir=dest_dir

  for dir in origin_faces_paths:
    video_id=dir.split("/")[-1]
    try:
      emotion=dict_set[video_id]
      foto_files=glob.glob(dir+"/*")
      if test==False:
        #Train or val set
        if video_id in videos_val:
          print("j:",j)
          j_fotos=j_fotos+len(foto_files)
          print("valfotos: ",j_fotos)
          destination_dir=videos_val_dir
          j+=1
        elif video_id in videos_tr:
          print("k:",k)
          k_fotos=k_fotos+len(foto_files)
          destination_dir=dest_dir
          print("trainfotos: ",k_fotos)
          k+=1
     
      
      print("destination dir", destination_dir)
      for foto_file in foto_files:
        foto_id=foto_file.split("/")[-1]

        if version1==True:
          if not os.path.exists(destination_dir+emotion+"/"+video_id):
            os.makedirs(destination_dir+emotion+"/"+video_id)
          copyfile(foto_file,destination_dir+emotion+"/"+video_id+"/"+foto_id)
        else:
          copyfile(foto_file,destination_dir+emotion+"/"+video_id+"_"+foto_id)
    except Exception as e:
      print(e)
      print("label error with video id: ",video_id)
      print("i:",i)
      i=i+1

# train set:
print("TRAINING SET: ")
origin_faces_paths=glob.glob("/content/Train_faces_videos/Faces/*")
destination_dir="/content/EMOTIW_train_Faces/"
tidy_set(origin_faces_paths,destination_dir,train_video_labels,version1=True)

#test set:
print("---------------------------------------------------------")
print("VAL SET: ")
origin_faces_paths=glob.glob("/content/Val_faces_videos/Faces/*")
# origin_faces_paths=glob.glob("/content/Faces_val_videos/*")
destination_dir="/content/EMOTIW_test_Faces/"
tidy_set(origin_faces_paths,destination_dir,val_video_labels,version1=True,test=True)

In [ ]:
def evaluate_model(model):
  datagen = ImageDataGenerator(rescale=1. / 255)
  test_set = datagen.flow_from_directory(
      '/content/EMOTIW_test_Faces',
      target_size=(img_width, img_height),
      batch_size=128,
      class_mode='categorical',
      shuffle=False)

  predictions=model.predict(test_set)
  print(test_set.class_indices.keys())
  print('Confusion Matrix')
  ypred=[round(list(i).index(max(i))) for i in predictions]
  print(confusion_matrix(test_set.classes, ypred)[:,:])
  # print(ypred)
  print("accuracy: ",accuracy_score(test_set.classes, ypred))
  for i in range(0,6):
    # print("Train: ",list(train_set.class_indices.keys())[i]," ",list(train_set.labels).count(i))
    print("Test: ",list(test_set.class_indices.keys())[i]," ",list(test_set.labels).count(i))

  print("evaluate: ",model.evaluate_generator(test_set))
  print(test_set.class_indices)
  x,y = test_set.next()
  print(len(x))
  import random
  for i in range(15):
      j=random.randint(1,127)
      image = x[j]
      plt.imshow( tf.squeeze(image),cmap='gray')
      print("real",get_emo(y[j]))
      # print(ypred[i])
      print("predicted:",emotions_dict[ypred[i]])
      plt.show()

  return predictions

In [ ]:
# train set:
print("TRAINING SET: ")
origin_faces_paths=glob.glob("/content/Train_faces_videos/Faces/*")
destination_dir="/content/EMOTIW_train_Faces/"
tidy_set(origin_faces_paths,destination_dir,train_video_labels,version1=True)

#test set:
print("---------------------------------------------------------")
print("VAL SET: ")
# origin_faces_paths=glob.glob("/content/Val_faces_videos/Faces/*")
origin_faces_paths=glob.glob("/content/Faces_val_videos/*")
destination_dir="/content/EMOTIW_val_Faces/"
tidy_set(origin_faces_paths,destination_dir,val_video_labels,version1=True)

In [ ]:
videos_paths=glob.glob("/content/EMOTIW_train_Faces/*/*")
y_real_afew_tr, y_pred_afew_tr, predictions_tr, afew_predictions_tr =evaluate_videos_score(videos_paths,model)

videos_paths=glob.glob("/content/EMOTIW_val_Faces/*/*")
y_real_afew_val, y_predafew_val, predictions_val, afew_predictions_val=evaluate_videos_score(videos_paths,model)

videos_paths=glob.glob("/content/EMOTIW_test_Faces/*/*")
y_real_afew_te, y_predafew_te, predictions_te, afew_predictions_te=evaluate_videos_score(videos_paths,model)

In [ ]:
import pickle

a_file = open("/content/gdrive/MyDrive/TFM_MUSI/combined_model/afew_videos_imgs_predictions_m2_tr.pkl", "wb")

pickle.dump(afew_predictions_tr, a_file)

a_file.close()

a_file = open("/content/gdrive/MyDrive/TFM_MUSI/combined_model/afew_videos_imgs_predictions_m2_te.pkl", "wb")

pickle.dump(afew_predictions_te, a_file)


a_file = open("/content/gdrive/MyDrive/TFM_MUSI/combined_model/afew_videos_imgs_predictions_m2_val.pkl", "wb")

pickle.dump(afew_predictions_val, a_file)

a_file.close()